In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
HOME = '/srv/home/christinedk/wp_internship/'
DATA_DIR = HOME + 'data/'
sys.path.append(HOME + 'collaboration/')

In [3]:
import pandas as pd
import json
import numpy as np
from collections import Counter

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, confusion_matrix, f1_score
from sklearn.preprocessing import StandardScaler

from math import log2
from utils import entropy

/usr/lib/python3/dist-packages/sklearn/linear_model/least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.float).eps,
/usr/lib/python3/dist-packages/sklearn/linear_model/least_angle.py:597: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.float).eps, copy_X=True, fit_path=True,
/usr/lib/python3/dist

In [4]:
# Page history

In [5]:
template='fanpov'

In [6]:
with open(HOME+'features/activity_{}.json'.format(template),'rb') as f:
    features_pos = json.load(f)
with open(HOME+'negative_features/activity_{}.json'.format(template),'rb') as f:
    features_neg = json.load(f)

In [7]:
len(features_pos)

721

In [8]:
len(features_neg)

567

In [9]:
article_features_neg = pd.DataFrame([d['article'] for d in features_neg])
article_features_neg['label'] = 0
article_features_pos = pd.DataFrame([d['article'] for d in features_pos])
article_features_pos['label'] = 1

In [10]:
article_data = pd.concat([article_features_neg, article_features_pos])
labels = article_data['label']
feat = article_data.drop('label',axis=1)

In [11]:
len(article_data)

1288

In [12]:
correlations = feat.corrwith(labels).reset_index().rename(columns={'index':'feature',0:'score'})
correlations['abs'] = correlations.score.abs()
correlations.sort_values(by='abs', ascending=False)[:10]

,feature,score,abs
8,frac_recent_revisions,0.203390,0.203390
14,recent_response_time,0.141511,0.141511
1,time_to_respond,0.115258,0.115258
11,concentration_ratio,0.101302,0.101302
12,contribution_frac_entropy,-0.089289,0.089289
10,frac_anon_revisions,-0.085547,0.085547
13,recent_edit_size,0.081533,0.081533
7,num_editors,-0.060535,0.060535
6,num_revisions,-0.060449,0.060449
9,top_contributor_frac,0.057898,0.057898


In [13]:
# linear model

In [112]:
hypers = {'solver':'lbfgs','max_iter':10000,'C':10000}

In [113]:
scaler = StandardScaler()
model = LogisticRegression(**hypers)
#C=0.1,class_weight={0:1,1:5}

X_train, X_test, y_train, y_test = train_test_split(feat.fillna(0),labels)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

/usr/lib/python3/dist-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/lib/python3/dist-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/christinedk/venv/lib/python3.7/site-packages/ipykernel_launcher.py:7: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  import sys


In [114]:
model.fit(X=X_train,y=y_train)
predictions=model.predict_proba(X_test)
y_pred = np.argmax(predictions,axis=1)
roc_auc_score(y_score=predictions[:,1],y_true=y_test)

0.4720271535580524

In [115]:
confusion_matrix(y_pred=y_pred,y_true=y_test)

array([[ 50,  94],
       [ 73, 105]])

In [116]:
f1_score(y_pred=y_pred, y_true=y_test)

0.5570291777188329

In [117]:
# user-article

In [118]:
def aggregate_features(article_users):
    if len(article_users) == 0:
        return {}
    else:
        article_users = pd.DataFrame(article_users).drop('event_user_id',axis=1)
        means = {'mean_'+key:val for key, val in article_users.mean().items()}
        std = {'std_'+key:val for key, val in article_users.std().items()}
        max_ = {'max_'+key:val for key, val in article_users.max().items()}
        ent = {'frac_page_edits_ent': entropy(article_users.frac_page_edits)}
        features = {**means, **std, **max_, **ent}

        return features

In [119]:
user_article_features_pos = pd.DataFrame([aggregate_features(d['user_article']) 
                                          for d in features_pos])
user_article_features_neg = pd.DataFrame([aggregate_features(d['user_article']) 
                                          for d in features_neg])

In [120]:
user_article_features_pos['label'] = 1
user_article_features_neg['label'] = 0

user_article_data = pd.concat([user_article_features_neg,user_article_features_pos])
labels = user_article_data['label']
feat = user_article_data.drop('label',axis=1)

In [121]:
correlations = feat.corrwith(labels).reset_index().rename(columns={'index':'feature',0:'score'})
correlations['abs'] = correlations.score.abs()
correlations.sort_values(by='abs', ascending=False)[:10]

,feature,score,abs
5,mean_frac_page_edits,0.133593,0.133593
2,mean_time_to_respond,0.123527,0.123527
10,std_time_to_respond,0.120352,0.120352
3,mean_time_responded_to,0.110808,0.110808
11,std_time_responded_to,0.091923,0.091923
18,max_time_to_respond,0.090027,0.090027
13,std_frac_page_edits,0.074329,0.074329
21,max_frac_page_edits,0.071453,0.071453
24,frac_page_edits_ent,-0.066356,0.066356
15,std_revision_is_identity_revert,-0.060916,0.060916


In [122]:
scaler = StandardScaler()
model = LogisticRegression(**hypers)
#C=0.1,class_weight={0:1,1:5}

X_train, X_test, y_train, y_test = train_test_split(feat.fillna(0),labels)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [123]:
model.fit(X=X_train,y=y_train)
predictions=model.predict_proba(X_test)
y_pred = np.argmax(predictions,axis=1)
roc_auc_score(y_score=predictions[:,1],y_true=y_test)

0.585677648451371

In [124]:
confusion_matrix(y_pred=y_pred,y_true=y_test)

array([[ 56,  81],
       [ 59, 126]])

In [125]:
f1_score(y_pred=y_pred, y_true=y_test)

0.6428571428571429

In [126]:
# talk vol

In [127]:
with open(HOME+'features/talk_{}.json'.format(template),'rb') as f:
    talk_pos = json.load(f)
with open(HOME+'negative_features/talk_{}.json'.format(template),'rb') as f:
    talk_neg = json.load(f)

In [128]:
talk_vol_neg = pd.DataFrame([d['talk_volume'] for d in talk_neg])
talk_vol_neg['label'] = 0
talk_vol_pos = pd.DataFrame([d['talk_volume'] for d in talk_pos])
talk_vol_pos['label'] = 1

In [129]:
talk_vol = pd.concat([talk_vol_neg,talk_vol_pos])
labels = talk_vol['label']
feat = talk_vol.drop('label',axis=1)

In [130]:
correlations = feat.corrwith(labels).reset_index().rename(columns={'index':'feature',0:'score'})
correlations['abs'] = correlations.score.abs()
correlations.sort_values(by='abs', ascending=False)[:10]

,feature,score,abs
5,mean_response_time,0.064148,0.064148
2,num_editors,-0.048739,0.048739
0,num_revisions,-0.032085,0.032085
4,mean_edit_size,-0.031951,0.031951
6,page_talk_ratio,-0.029191,0.029191
1,frac_recent_revisions,0.021272,0.021272
3,top_contributor_frac,0.019637,0.019637


In [131]:
scaler = StandardScaler()
model = LogisticRegression(**hypers)
#C=0.1,class_weight={0:1,1:5}

X_train, X_test, y_train, y_test = train_test_split(feat.fillna(0),labels)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

/usr/lib/python3/dist-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/lib/python3/dist-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/christinedk/venv/lib/python3.7/site-packages/ipykernel_launcher.py:7: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  import sys


In [132]:
model.fit(X=X_train,y=y_train)
predictions=model.predict_proba(X_test)
y_pred = np.argmax(predictions,axis=1)
roc_auc_score(y_score=predictions[:,1],y_true=y_test)

0.5274322169059011

In [133]:
confusion_matrix(y_pred=y_pred,y_true=y_test)

array([[ 23, 109],
       [ 39, 151]])

In [134]:
f1_score(y_pred=y_pred, y_true=y_test)

0.6711111111111111

In [135]:
# talk lang

In [136]:
with open(HOME+'features/talk_{}.json'.format(template),'rb') as f:
    talk_pos = json.load(f)
with open(HOME+'negative_features/talk_{}.json'.format(template),'rb') as f:
    talk_neg = json.load(f)

In [137]:
def aggregate_language(talk_feat):
    if len(talk_feat) == 0:
        return {}
    else:
        all_utts = pd.DataFrame(list(np.concatenate(list(talk_feat.values()))))
        means = {'mean_'+key:val for key, val in all_utts.mean().items()}
        std = {'std_'+key:val for key, val in all_utts.std().items()}
        max_ = {'max_'+key:val for key, val in all_utts.max().items()}
        features = {**means, **std, **max_}

        return features

In [138]:
talk_lang_neg = pd.DataFrame([aggregate_language(d['talk_language']) for d in talk_neg])
talk_lang_neg['label'] = 0

In [139]:
len(talk_lang_neg)

567

In [140]:
talk_lang_pos = pd.DataFrame([aggregate_language(d['talk_language']) for d in talk_pos])
talk_lang_pos['label'] = 1

In [141]:
len(talk_lang_pos)

721

In [142]:
talk_lang = pd.concat([talk_lang_neg,talk_lang_pos])
labels = talk_lang['label']
feat = talk_lang.drop('label',axis=1)

In [143]:
correlations = feat.corrwith(labels).reset_index().rename(columns={'index':'feature',0:'score'})
correlations['abs'] = correlations.score.abs()
correlations.sort_values(by='abs', ascending=False)[:10]

,feature,score,abs
45,max_politeness_markers_==Please_start==,-0.050414,0.050414
49,max_politeness_markers_==Factuality==,-0.049970,0.049970
58,max_politeness_markers_==Indirect_(greeting)==,-0.047523,0.047523
54,max_politeness_markers_==1st_person==,-0.045786,0.045786
56,max_politeness_markers_==2nd_person==,-0.044170,0.044170
59,max_politeness_markers_==Direct_question==,-0.043029,0.043029
48,max_politeness_markers_==Hedges==,-0.042943,0.042943
46,max_politeness_markers_==HASHEDGE==,-0.042153,0.042153
44,max_politeness_markers_==Please==,-0.040334,0.040334
65,max_reply_depth,-0.040155,0.040155


In [144]:
scaler = StandardScaler()
model = LogisticRegression(**hypers)
#C=0.1,class_weight={0:1,1:5}

X_train, X_test, y_train, y_test = train_test_split(feat.fillna(0),labels)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [145]:
model.fit(X=X_train,y=y_train)
predictions=model.predict_proba(X_test)
y_pred = np.argmax(predictions,axis=1)
roc_auc_score(y_score=predictions[:,1],y_true=y_test)

0.3959935897435898

In [146]:
confusion_matrix(y_pred=y_pred,y_true=y_test)

array([[ 12, 118],
       [ 33, 159]])

In [147]:
f1_score(y_pred=y_pred, y_true=y_test)

0.6780383795309168

In [148]:
# editors

In [149]:
def aggregate_editor(editor_feat):
    if len(editor_feat) == 0:
        return {}
    else:
        all_utts = pd.DataFrame(editor_feat).drop('event_user_id',axis=1)
        means = {'mean_'+key:val for key, val in all_utts.mean().items()}
        std = {'std_'+key:val for key, val in all_utts.std().items()}
        max_ = {'max_'+key:val for key, val in all_utts.max().items()}
        features = {**means, **std, **max_}

        return features

In [150]:
with open(HOME+'features/editors{}_v2.json'.format(template),'rb') as f:
    editors_pos = json.load(f)
    
with open(HOME+'negative_features/editors{}_v2.json'.format(template),'rb') as f:
    editors_neg = json.load(f)

In [152]:
editor_neg = pd.DataFrame([aggregate_editor(d['editor']) for d in editors_neg])
editor_neg['label'] = 0
editor_pos = pd.DataFrame([aggregate_editor(d['editor']) for d in editors_pos])
editor_pos['label'] = 1

In [153]:
len(editor_neg)

567

In [154]:
len(editor_pos)

721

In [155]:
editor_data = pd.concat([editor_neg,editor_pos])
labels = editor_data['label']
feat = editor_data.drop('label',axis=1)

In [156]:
correlations = feat.corrwith(labels).reset_index().rename(columns={'index':'feature',0:'score'})
correlations['abs'] = correlations.score.abs()
correlations.sort_values(by='abs', ascending=False)[:10]

,feature,score,abs
20,max_num_groups,-0.077194,0.077194
29,max_contribution_frac_entropy,-0.075266,0.075266
24,max_num_curr_blocks,-0.064384,0.064384
25,max_num_reverts_by_others,-0.059058,0.059058
26,max_num_reverts_of_others,-0.048963,0.048963
19,std_contribution_frac_entropy,0.042711,0.042711
28,max_talk_article_ratio,-0.040673,0.040673
14,std_num_curr_blocks,-0.040308,0.040308
15,std_num_reverts_by_others,-0.035276,0.035276
2,mean_num_edits,0.033765,0.033765


In [157]:
scaler = StandardScaler()
model = LogisticRegression(**hypers)
#C=0.1,class_weight={0:1,1:5}

X_train, X_test, y_train, y_test = train_test_split(feat.fillna(0),labels)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [158]:
model.fit(X=X_train,y=y_train)
predictions=model.predict_proba(X_test)
y_pred = np.argmax(predictions,axis=1)
roc_auc_score(y_score=predictions[:,1],y_true=y_test)

0.49715324046032705

In [159]:
confusion_matrix(y_pred=y_pred,y_true=y_test)

array([[ 37,  90],
       [ 69, 126]])

In [160]:
f1_score(y_pred=y_pred, y_true=y_test)

0.6131386861313869

In [161]:
# editor-editor

In [162]:
def aggregate(df):
    means = {'mean_'+key:val for key, val in df.mean().items()}
    std = {'std_'+key:val for key, val in df.std().items()}
    max_ = {'max_'+key:val for key, val in df.max().items()}
    features = {**means, **std, **max_}
    return features

def aggregate_collab(editor_feat):
    if len(editor_feat['undirected']) == 0:
        features = {}
        
    else:
        directed = pd.DataFrame(editor_feat['directed']).drop(['event_user_id','event_user_id_r'],axis=1)
        undirected = pd.DataFrame(editor_feat['undirected']).drop('pair',axis=1)

        dir_features = aggregate(directed)
        undir_features = aggregate(undirected)

        features = {**dir_features,**undir_features}

    return features

In [163]:
collab_neg = pd.DataFrame([aggregate_collab(d['collaboration']) for d in editors_neg])
collab_neg['label'] = 0
collab_pos = pd.DataFrame([aggregate_collab(d['collaboration']) for d in editors_pos])
collab_pos['label'] = 1

In [164]:
collab_data = pd.concat([editor_neg,editor_pos])
labels = collab_data['label']
feat = collab_data.drop('label',axis=1)

In [165]:
correlations = feat.corrwith(labels).reset_index().rename(columns={'index':'feature',0:'score'})
correlations['abs'] = correlations.score.abs()
correlations.sort_values(by='abs', ascending=False)[:10]

,feature,score,abs
20,max_num_groups,-0.077194,0.077194
29,max_contribution_frac_entropy,-0.075266,0.075266
24,max_num_curr_blocks,-0.064384,0.064384
25,max_num_reverts_by_others,-0.059058,0.059058
26,max_num_reverts_of_others,-0.048963,0.048963
19,std_contribution_frac_entropy,0.042711,0.042711
28,max_talk_article_ratio,-0.040673,0.040673
14,std_num_curr_blocks,-0.040308,0.040308
15,std_num_reverts_by_others,-0.035276,0.035276
2,mean_num_edits,0.033765,0.033765


In [166]:
scaler = StandardScaler()
model = LogisticRegression(**hypers)
#C=0.1,class_weight={0:1,1:5}

X_train, X_test, y_train, y_test = train_test_split(feat.fillna(0),labels)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [167]:
model.fit(X=X_train,y=y_train)
predictions=model.predict_proba(X_test)
y_pred = np.argmax(predictions,axis=1)
roc_auc_score(y_score=predictions[:,1],y_true=y_test)

0.5420958177278401

In [168]:
confusion_matrix(y_pred=y_pred,y_true=y_test)

array([[ 35, 109],
       [ 51, 127]])

In [169]:
f1_score(y_pred=y_pred, y_true=y_test)

0.6135265700483091

In [170]:
# combined

In [171]:
labels = article_data['label']

feat = pd.concat([ds.reset_index(drop=True).drop('label',axis=1) 
                  for ds in [article_data,talk_vol,talk_lang,user_article_data,editor_data,collab_data]],axis=1)

In [172]:
len(feat)

1288

In [173]:
correlations = feat.corrwith(labels).reset_index().rename(columns={'index':'feature',0:'score'})
correlations['abs'] = correlations.score.abs()
correlations.sort_values(by='abs', ascending=False)[:10]

,feature,score,abs
160,std_time_since_registration,-0.089352,0.089352
130,std_time_since_registration,-0.089352,0.089352
143,mean_num_groups,-0.085850,0.085850
113,mean_num_groups,-0.085850,0.085850
2,article_age_years,0.078321,0.078321
153,std_num_groups,-0.073301,0.073301
123,std_num_groups,-0.073301,0.073301
12,contribution_frac_entropy,0.068298,0.068298
10,frac_anon_revisions,0.060782,0.060782
33,mean_politeness_markers_==1st_person_start==,0.060210,0.060210


In [174]:
scaler = StandardScaler()
model = LogisticRegression(**hypers)#,class_weight={0:1,1:5}

X_train, X_test, y_train, y_test = train_test_split(feat.fillna(0),labels)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

/usr/lib/python3/dist-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/lib/python3/dist-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/christinedk/venv/lib/python3.7/site-packages/ipykernel_launcher.py:6: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  


In [175]:
model.fit(X=X_train,y=y_train)
predictions=model.predict_proba(X_test)
y_pred = np.argmax(predictions,axis=1)
roc_auc_score(y_score=predictions[:,1],y_true=y_test)

0.48021978021978023

In [176]:
confusion_matrix(y_pred=y_pred,y_true=y_test)

array([[ 49,  91],
       [ 77, 105]])

In [177]:
f1_score(y_pred=y_pred, y_true=y_test)

0.5555555555555555

In [178]:
# Thoughts:

# article:
# comparing to same pages means some features are less relevant; eg. num_editors, mean edit_size, top_contributor_frac, etc
# recency features should be more important 
#     - calculate all features for recent revisions only as well?


# user-article
# empty for some negatives? sample page': 189559,'date': '2003-02-28 16:09:01'
# revert features are incorrect, need to re-export

In [ ]:
# Editor